# Notebook Title

## Setup Python and R environment
you can ignore this section

In [1]:
%load_ext rpy2.ipython
%load_ext autoreload
%autoreload 2

%matplotlib inline  
from matplotlib import rcParams
rcParams['figure.figsize'] = (16, 100)

import warnings
from rpy2.rinterface import RRuntimeWarning
warnings.filterwarnings("ignore") # Ignore all warnings
# warnings.filterwarnings("ignore", category=RRuntimeWarning) # Show some warnings

import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
from IPython.display import display, HTML

In [2]:
%%javascript
// Disable auto-scrolling
IPython.OutputArea.prototype._should_scroll = function(lines) {
    return false;
}

<IPython.core.display.Javascript object>

In [3]:
%%R

# My commonly used R imports

require('tidyverse')

── Attaching core tidyverse packages ──────────────────────── tidyverse 2.0.0 ──
✔ dplyr     1.1.4     ✔ readr     2.1.5
✔ forcats   1.0.0     ✔ stringr   1.5.1
✔ ggplot2   3.5.1     ✔ tibble    3.2.1
✔ lubridate 1.9.4     ✔ tidyr     1.3.1
✔ purrr     1.0.4     
── Conflicts ────────────────────────────────────────── tidyverse_conflicts() ──
✖ dplyr::filter() masks stats::filter()
✖ dplyr::lag()    masks stats::lag()
ℹ Use the conflicted package (<http://conflicted.r-lib.org/>) to force all conflicts to become errors


Loading required package: tidyverse


## 👉 load census data

You can write code here to download your dataset. Or if you already have it, just leave the URL in the comments and just load it into a pandas or R (or both) dataframe.

In [4]:
%%R

library(tidycensus)

census_api_key("9b315cfaf570bf06e8206cb8cde6b5a0dc021c7b")

To install your API key for use in future sessions, run this function with `install = TRUE`.


In [5]:
%%R

county_income <- get_acs(
  geography = "county",
  variables = "S1901_C01_012E",
  year = 2022,
  survey = "acs5"
)

head(county_income)

# A tibble: 6 × 5
  GEOID NAME                    variable      estimate   moe
  <chr> <chr>                   <chr>            <dbl> <dbl>
1 01001 Autauga County, Alabama S1901_C01_012    68315  4941
2 01003 Baldwin County, Alabama S1901_C01_012    71039  2374
3 01005 Barbour County, Alabama S1901_C01_012    39712  3289
4 01007 Bibb County, Alabama    S1901_C01_012    50669  8260
5 01009 Blount County, Alabama  S1901_C01_012    57440  3308
6 01011 Bullock County, Alabama S1901_C01_012    36136  4731


Getting data from the 2018-2022 5-year ACS
Using the ACS Subject Tables


In [6]:
%%R

county_income <- county_income %>%
  select(GEOID, county = NAME, median_income = estimate)

head(county_income)


# A tibble: 6 × 3
  GEOID county                  median_income
  <chr> <chr>                           <dbl>
1 01001 Autauga County, Alabama         68315
2 01003 Baldwin County, Alabama         71039
3 01005 Barbour County, Alabama         39712
4 01007 Bibb County, Alabama            50669
5 01009 Blount County, Alabama          57440
6 01011 Bullock County, Alabama         36136


 ## 👉 Load Non Census Data


In [7]:
%%R

if (!require(openxlsx)) {
  install.packages("openxlsx")
}
library(openxlsx)
library(dplyr)

file_path <- "/Users/luciadelatorre/Desktop/COLUMBIA/GitHub/connect-to-census/healthrankingsperstate2022.xlsx"

health_data <- read.xlsx(file_path, sheet = "Select Measure Data")

head(health_data)

     X1      X2      X3 Premature.Death     X5
1  FIPS   State  County      Unreliable Deaths
2 01000 Alabama    <NA>            <NA> 102760
3 01001 Alabama Autauga            <NA>   1008
4 01003 Alabama Baldwin            <NA>   3944
5 01005 Alabama Barbour            <NA>    587
6 01007 Alabama    Bibb            <NA>    509
                                 X6                 X7                 X8
1 Years of Potential Life Lost Rate       95% CI - Low      95% CI - High
2                      11853.247248        11744.01482 11962.479676000001
3                9938.2633822999996 9021.2971328999993       10855.229632
4                8957.1126858999996 8499.3392232999995 9414.8861484999998
5                      12738.656137 11133.499024999999 14343.813249999999
6                      11708.948038 10166.716823000001       13251.179253
                    X9                              X10
1     National Z-Score YPLL Rate (Hispanic (all races))
2                 <NA>                   

Loading required package: openxlsx


In [8]:
%%R

colnames(health_data)

  [1] "X1"                                "X2"                               
  [3] "X3"                                "Premature.Death"                  
  [5] "X5"                                "X6"                               
  [7] "X7"                                "X8"                               
  [9] "X9"                                "X10"                              
 [11] "X11"                               "X12"                              
 [13] "X13"                               "X14"                              
 [15] "X15"                               "X16"                              
 [17] "X17"                               "X18"                              
 [19] "X19"                               "X20"                              
 [21] "X21"                               "X22"                              
 [23] "X23"                               "X24"                              
 [25] "X25"                               "X26"                 

In [9]:
%%R
health_data <- health_data %>%
  slice(-(1:1)) %>%
  select(X1, X2, X3, Poor.Physical.Health.Days) %>%
  rename(FIPS = X1, State = X2, County = X3, UnhealthyDaysMonth = Poor.Physical.Health.Days) %>%
  filter(!is.na(FIPS) & FIPS != "FIPS" & !is.na(County))

health_data$UnhealthyDaysMonth <- as.numeric(health_data$UnhealthyDaysMonth)

head(health_data)

   FIPS   State  County UnhealthyDaysMonth
1 01001 Alabama Autauga           4.300364
2 01003 Alabama Baldwin           3.876436
3 01005 Alabama Barbour           5.431030
4 01007 Alabama    Bibb           5.055619
5 01009 Alabama  Blount           4.802400
6 01011 Alabama Bullock           5.814122


 ## 👉 Merge datasets


In [10]:
%%R
merged_data <- county_income %>%
  left_join(health_data, by = c("GEOID" = "FIPS"))
head(merged_data)

# A tibble: 6 × 6
  GEOID county                  median_income State   County  UnhealthyDaysMonth
  <chr> <chr>                           <dbl> <chr>   <chr>                <dbl>
1 01001 Autauga County, Alabama         68315 Alabama Autauga               4.30
2 01003 Baldwin County, Alabama         71039 Alabama Baldwin               3.88
3 01005 Barbour County, Alabama         39712 Alabama Barbour               5.43
4 01007 Bibb County, Alabama            50669 Alabama Bibb                  5.06
5 01009 Blount County, Alabama          57440 Alabama Blount                4.80
6 01011 Bullock County, Alabama         36136 Alabama Bullock               5.81


In [11]:
%%R
merged_data <- merged_data %>%
  select(GEOID, median_income, State, County, UnhealthyDaysMonth)

head(merged_data)

# A tibble: 6 × 5
  GEOID median_income State   County  UnhealthyDaysMonth
  <chr>         <dbl> <chr>   <chr>                <dbl>
1 01001         68315 Alabama Autauga               4.30
2 01003         71039 Alabama Baldwin               3.88
3 01005         39712 Alabama Barbour               5.43
4 01007         50669 Alabama Bibb                  5.06
5 01009         57440 Alabama Blount                4.80
6 01011         36136 Alabama Bullock               5.81


 ## 👉 Save merged dataset

In [12]:
%%R
write.csv(merged_data, "/Users/luciadelatorre/Desktop/COLUMBIA/GitHub/connect-to-census/county_health_income.csv", row.names = FALSE)